In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

we are visualising the data 


In [5]:
bangres=pd.read_csv('../input/zomato-bangalore-restaurants/zomato.csv')


In [6]:
bangres.head()

In [7]:
bangres.columns

In [8]:
##removing unwanted columns in bangres table
bangres=bangres.drop(['url','address','book_table','phone','reviews_list', 'menu_item','dish_liked'],axis=1)

In [9]:
bangres.head()

In [84]:
##heatmap
sns.heatmap(bangres.isnull(),yticklabels=False,cbar=False,cmap='viridis')
##the below heatmap will how the null values

In [10]:
bangres.describe()

In [11]:
bangres.shape

In [12]:
bangres.info()

In [13]:
##removing duplicates
bangres.drop_duplicates(inplace=True)
bangres.shape

In [14]:
bangres.head()
##unique values in rates 
bangres['rate'].unique

In [15]:
##cleaning rates
def remove(val):
    if(val=='NEW' or val=='-'):
        return np.nan
    else:
        val=str(val).split('/')
        val=val[0]
        return float(val)
    
      
bangres['rate']=bangres['rate'].apply(remove)

In [16]:
##checking whether /5 and - is removed or not 
bangres.head()

In [17]:
##checking null values in rate
bangres.rate.isnull().sum()

In [18]:
##removing the null in rate 
bangres['rate'].fillna(bangres['rate'].mean(),inplace=True)
##checking whether nulls are removed 
bangres.rate.isnull().sum()

In [19]:
bangres.info()
bangres.shape
bangres.dropna(inplace=True)
bangres.head()

In [20]:
##renaming some columns
bangres.rename(columns={'approx_cost(for two people)':'cost_of_2','listed_in(type)':'restraunt_type'},inplace=True)
bangres.head()
bangres.rename(columns={'restraunt_type':'service_type'},inplace=True)
bangres.head()

In [21]:
#cleaning location 
bangres['location'].unique()

In [22]:
bangres.columns

In [23]:
bangres.drop(['listed_in(city)'],axis=1,inplace=True)
bangres.head()

In [24]:
bangres['cost_of_2'].unique()

In [25]:
##removing ,'s from the column cost_of_2
def change(cost):
    cost=str(cost)
    if ',' in cost:
        cost=cost.replace(',','')
        return float(cost)
    else:
        return float(cost)
    
bangres['cost_of_2']=bangres['cost_of_2'].apply(change)
bangres['cost_of_2'].head()
bangres.head()

In [26]:
bangres['rest_type'].unique()

In [27]:
bangres['rest_type'].value_counts()


In [28]:
rest_types=bangres['rest_type'].value_counts(ascending=False)
rest_types

In [29]:
bangres.head()

In [30]:
##Grouping restraunts having less than 100
rest_types_lessthan_1000=rest_types[rest_types<1000]
rest_types_lessthan_1000

In [31]:
##grouping
def changerest_types(rest):
    if(rest in rest_types_lessthan_1000):
        return 'others'
    else:
        return rest
bangres['rest_type']=bangres['rest_type'].apply(changerest_types)
bangres['rest_type'].head()

In [32]:
bangres['rest_type'].value_counts()

##location cleaning from bangres(bangalore restraunts) 

In [33]:
bangres['location'].value_counts()
##selecting the restraunts having less than 300 locations
location=bangres['location'].value_counts(ascending=False)
location_lessthan_300=location[location<300]
def changelocation(loc):
    if(loc in location_lessthan_300):
        return 'others'
    else:
        return loc
bangres['location']=bangres['location'].apply(changelocation)
bangres['location'].value_counts()

In [34]:
##CLEANING CUISINES WHICH ARE HAVING <100
cuisine=bangres['cuisines'].value_counts(ascending=False)
cuisine_lessthan_100=cuisine[cuisine<100]

def changecuisines (cus):
    if(cus in cuisine_lessthan_100):
        return 'others'
    else:
        return cus
bangres['cuisines']=bangres['cuisines'].apply(changecuisines)
bangres['cuisines'].value_counts()


In [35]:
bangres['service_type'].value_counts()

# visualisations begin

In [36]:
##showing the number of restraunts as per locations
plt.figure(figsize=(16,10))
ax=sns.countplot(bangres['location'])
plt.xticks(rotation=90)


In [37]:
##showing the online orders 
plt.figure(figsize=(5,5))
onlinebookings=bangres['online_order']
sns.countplot(onlinebookings,palette='magma')

In [38]:
##displaying the ratings in online orders
plt.figure(figsize=(7,7))
sns.boxplot(x='online_order',y='rate',data=bangres)

In [39]:
##displaying the cost of two people with location 
plt.figure(figsize=(40,40))
sns.barplot(x=bangres['location'],y=bangres['cost_of_2'])

In [40]:
##displaying the ratings as per type of the restraunt
plt.figure(figsize=(15,10))
sns.boxplot(x=bangres['service_type'],y=bangres['rate'],palette='hot')

In [41]:
df1 = bangres.groupby(['location','online_order'])['name'].count()
df1.to_csv('location_online.csv')
df1 = pd.read_csv('location_online.csv')
df1 = pd.pivot_table(df1, values=None, index=['location'], columns=['online_order'], fill_value=0, aggfunc=np.sum)
df1

In [42]:
df1.plot(kind = 'bar', figsize = (15,8))

In [46]:
bangres.head()

In [50]:
df2 = bangres.groupby(['location','rest_type'])['name'].count()
df2.to_csv('location_Type.csv')
df2 = pd.read_csv('location_Type.csv')
df2 = pd.pivot_table(df2, values=None, index=['location'], columns=['rest_type'], fill_value=0, aggfunc=np.sum)
df2

In [52]:
df2.plot(kind = 'bar', figsize = (40,12))

In [58]:
votes_table= bangres[['location', 'votes']]
votes_table.drop_duplicates()
df3 = votes_table.groupby(['location'])['votes'].sum()
df3 = df3.to_frame()
df3 = df3.sort_values('votes', ascending=False)
df3.head()

In [59]:
plt.figure(figsize = (20,10))
sns.barplot(df3.index , df3['votes'])
plt.xticks(rotation = 90)

In [64]:
#top cuisines
df4 = bangres[['cuisines', 'votes']]
df4.drop_duplicates()
cuisine_table = df4.groupby(['cuisines'])['votes'].sum()
cuisine_table = cuisine_table.to_frame()
cuisine_table = cuisine_table.sort_values('votes', ascending=False)
#removing the others restraunts with others cuisines
cuisine_table=cuisine_table.iloc[1:,:]
#display the top five cuisine according to customer 
cuisine_table.head()

In [65]:
plt.figure(figsize = (20,10))
sns.barplot(cuisine_table.index , cuisine_table['votes'])
plt.xticks(rotation = 90)

In [66]:
bangres.head()

In [77]:
##what will be the cost of dining 2 people as service_type
#top cuisines
df5 = bangres[[ 'service_type','cost_of_2']]
df5.drop_duplicates()
costof2_table = df5.groupby(['service_type'])['cost_of_2'].mean()
costof2_table = costof2_table.to_frame()
costof2_table = costof2_table.sort_values('cost_of_2', ascending=False)
#removing the others restraunts with others cuisines
costof2_table=costof2_table.iloc[1:,:]
#display the top five cuisine according to customer 
costof2_table.head()



In [81]:
plt.figure(figsize = (20,10))
sns.barplot(costof2_table.index , costof2_table['cost_of_2'])
